# Task 3 - Feature Matching using SIFT

Write a function which takes an image from the same dataset for training
and testing as in the previous task.

**Main steps:**
1. You first extract keypoints and feature descriptors from your
Test and Train images using standard SIFT or SURF feature extraction
function from a library.
2. Then you match features between images which will give you the raw noisy matches (correspondences).
3. Now you should decide which geometric transform to use to reject the outliers. (using RANSAC)
4. Finally, you will
define a score on the obtained inlier matches and will use this to detect the
objects (icons) scoring high for a given Test image. A basic score is counting
the inlier matches.

**Output:**

Detect objects in the Test images using SIFT or equivalent features (such as SURF), recognize to which class they belong, and identify
their scales and orientations. Similar as Task2, for visual demonstration the
function should open a box around each detected object and indicate its class
label. This box is scaled and rotated according to the object’s scale and orientation. Demonstrate example images(s) of the outcome detection in your report. Besides, demonstrate example images(s) that shows the feature-based
matches established between the recognised objects and a Test image, before
and after the outlier refinement step.

**Evaluation:**

Evaluate your algorithm on all Test images to report the overall Intersection over Union (IoU), False Positive (FPR), True Positive (TPR) and
Accuracy (ACC) rates, as well as the average runtime. Refer to the following report http://host.robots.ox.ac.uk/pascal/VOC/voc2012/devkit_
doc.pdf section 4.4 for further information about the evaluation metrics.
Show and explain cases where this scheme finds difficulty to perform correctly. Compare the SIFT/SURF results to that of Task2 algorithm e.g.,
does it improve the overall speed or accuracy? How much? Why?

**Hyperparameter tuning:**

Similarly, you will have some hyper-parameters to tune. This includes the
number of Octaves and the (within-octave) Scalelevels within SIFT to build
scale-spaces for keypoint detection, and the MaxRatio parameter within the
matchFeatures function to reject weak matches. How are these parameters
set for this task? Show quantitatively why.

**Notes:**

For task 2 and task 3, you are allowed to use library functions for creating the pyramid or using Gaussian convolution. You are also allowed to use the library functions for extracting features, for e.g. extracting SIFT features. You are allowed to use math libraries, for instance svd functions for computing the homography.

You are *not* allowed to use the `cv2.matchTemplate` or `cv2.BFMatcher`.
- Basically functions for matching features need to be coded. 
- You would need to implement RANSAC also yourself.

### Imports & Constants

In [47]:
%load_ext autoreload
%autoreload 2

import task3
import logging
import cv2 as cv
import pandas as pd
import numpy as np
from typing import Dict, Tuple

from pathlib import Path
from task3 import ImageDataset, ObjectDetector, Verbosity, TutorialObjectDetector

QUERY_IMG_DIR = Path("IconDataset", "png")
TEST_IMG_DIR = Path("Task3Dataset", "images")

ANNOTATIONS_DIR = Path("Task3Dataset", "annotations")

# Configure basic logging
logging.basicConfig(level=logging.INFO, format='[%(asctime)s]::[%(levelname)s] %(message)s')
logger = logging.getLogger(__name__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Run detection pipeline on dataset

In [48]:
def detect_on_dataset(
        test_imgs: ImageDataset, 
        query_imgs: ImageDataset, 
        sift_hps: Dict = {},
        ransac_hps: Dict = {},
        lowe_threshold: float = 0.7,
        min_match_count: int = 10, 
        verbose: Verbosity = Verbosity.MEDIUM
    ) -> Tuple[float, ...]:

    acc_lst, tpr_list, fpr_lst, fnr_lst = [], [], [], []
    detector = ObjectDetector(query_imgs, sift_hps, verbose=False, ransac_hyperparams=ransac_hps)

    # Iterate through each test image and detect objects in it. Compare these detctions to the ground truth annotations.
    for img, img_path in test_imgs:
        annotations_path = ANNOTATIONS_DIR / img_path.with_suffix(".csv").name
        img_annotations = pd.read_csv(annotations_path)

        print("\n", flush=True); logger.info(f"Detecting objects in {img_path.stem}")
        detections = detector.detect(img, lowe_threshold, min_match_count, draw=False)

        acc, tpr, fpr, fnr = task3.evaluate_detections(detections, img_annotations)
        acc_lst.append(acc); tpr_list.append(tpr); fpr_lst.append(fpr); fnr_lst.append(fnr)

    return np.mean(acc_lst), np.mean(tpr_list), np.mean(fpr_lst), np.mean(fnr_lst)

In [ ]:
test_images = ImageDataset(TEST_IMG_DIR, file_ext="png")
query_images = ImageDataset(QUERY_IMG_DIR, file_ext="png")

acc, tpr, fpr, fnr = detect_on_dataset(test_images, query_images)
print(acc, tpr, fpr, fnr)

Mine: 0.5850000000000001 0.7416666666666666 0.2764285714285714 0.25833333333333336

Tutorial w/ my matcher: 0.6775 0.7575 0.12666666666666665 0.2425

### Hyperparameter Optimisation

We'll use the following objective function to measure performance over a range of hyperparameters for `SIFT`, `RANSAC`, Lowe's Test, and minimum match counts. Then, using Bayesian optimisation, we'll minimise the function, hence we use '-accuracy'.

In [50]:
init_params = [
    128, 
    5,
    0.01,
    0.1,
    3,
    5.0,
    4,
    0,
    0.7,
    10,
]

In [51]:
import json

from tqdm.auto import tqdm
from skopt import gp_minimize
from skopt.space import Integer, Real
from skopt.utils import use_named_args
from skopt.callbacks import Callable


class NumpyEncoder(json.JSONEncoder):
    """Special json encoder for numpy types."""
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)
    

class LogDisplayProgressCallback(Callable):

    def __init__(self, tqdm_obj, hyperparam_names: list):
        self.tqdm_obj = tqdm_obj
        self.hyperparam_names = hyperparam_names
        self.iteration = 1

    def __call__(self, result):
        self.tqdm_obj.update(1)

        # Since we are minimising negative accuracy.
        best_score = -result.fun 
        current_params = result.x
        
        result_data = {
            'iteration': self.iteration,
            'best_score': best_score,
            'parameters': dict(zip(self.hyperparam_names, current_params))
        }

        with open('optimisation_log.json', 'a') as f:
            json.dump(result_data, f, cls=NumpyEncoder)
            f.write(',\n')

        self.iteration += 1


# Define the hyperparameter space
space = [
    Integer(100, 164, name='sift_n_features'),
    Integer(1, 10, name='sift_n_octave_layers'),
    Real(0.005, 0.2, name='sift_contrast_threshold'),
    Real(0.05, 20, name='sift_edge_threshold'),
    Real(0.1, 5.0, name='sift_sigma'),
    Real(1.0, 20.0, name='ransac_threshold'),
    Integer(4, 10, name='ransac_min_datapoints'),
    Integer(0, 8, name='ransac_inliers_threshold'),
    Real(0.5, 2.0, name='lowe_threshold'),
    Integer(4, 50, name='min_match_count'),
]

@use_named_args(space)
def objective_function(**params):
    sift_hps = {
        'nfeatures': params['sift_n_features'],
        'nOctaveLayers': params['sift_n_octave_layers'],
        'contrastThreshold': params['sift_contrast_threshold'],
        'edgeThreshold': params['sift_edge_threshold'],
        'sigma': params['sift_sigma'],
    }

    ransac_hps = {
        'inliers_threshold': params['ransac_inliers_threshold'],
        'min_datapoints': params['ransac_min_datapoints'],
        'reproj_threshold': params['ransac_threshold'],
    }

    acc, _, _, _ = detect_on_dataset(test_images, query_images, sift_hps, ransac_hps,
                                     params['lowe_threshold'],
                                     params['min_match_count'])
    
    # Negative because we minimise in the optimisation procedure.
    return -acc


ITERATIONS = 75

tqdm_o = tqdm(total=ITERATIONS)
callback = LogDisplayProgressCallback(tqdm_o, [param.name for param in space])

test_images = ImageDataset(TEST_IMG_DIR, file_ext="png")
query_images = ImageDataset(QUERY_IMG_DIR, file_ext="png")

result = gp_minimize(
    objective_function,
    dimensions=space,
    n_calls=ITERATIONS,
    random_state=41,
    x0=[init_params],
    callback=[callback]
)

tqdm_o.close()

  0%|          | 0/75 [00:00<?, ?it/s]

[2024-04-21 15:23:52,412]::[INFO] Detecting objects in test_image_16
[2024-04-21 15:23:52,736]::[INFO] Query image IconDataset/png/041-windmill.png yields enough good matches - 19/10. Finding transform...
[2024-04-21 15:23:53,136]::[INFO] Query image IconDataset/png/026-shop.png yields enough good matches - 12/10. Finding transform...
[2024-04-21 15:23:54,449]::[INFO] Query image IconDataset/png/048-hospital.png yields enough good matches - 13/10. Finding transform...
[2024-04-21 15:23:54,531]::[INFO] Query image IconDataset/png/024-fountain.png yields enough good matches - 17/10. Finding transform...
[2024-04-21 15:23:55,180]::[INFO] Query image IconDataset/png/003-bridge-1.png yields enough good matches - 13/10. Finding transform...
[2024-04-21 15:23:55,527]::[INFO] Query image IconDataset/png/044-ferris-wheel.png yields enough good matches - 12/10. Finding transform...
[2024-04-21 15:23:55,694]::[INFO] Query image IconDataset/png/030-telephone-booth.png yields enough good matches - 

[2024-04-21 15:23:56,537]::[INFO] Detecting objects in test_image_19
[2024-04-21 15:23:58,233]::[INFO] Query image IconDataset/png/015-barn.png yields enough good matches - 36/10. Finding transform...
[2024-04-21 15:23:59,105]::[INFO] Query image IconDataset/png/021-solar-panel.png yields enough good matches - 10/10. Finding transform...
[2024-04-21 15:23:59,944]::[INFO] Query image IconDataset/png/046-fire-station.png yields enough good matches - 17/10. Finding transform...


[2024-04-21 15:24:00,167]::[INFO] Detecting objects in test_image_18
[2024-04-21 15:24:00,656]::[INFO] Query image IconDataset/png/037-post-office.png yields enough good matches - 20/10. Finding transform...
[2024-04-21 15:24:00,811]::[INFO] Query image IconDataset/png/040-bus-stop.png yields enough good matches - 13/10. Finding transform...


[2024-04-21 15:24:04,413]::[INFO] Detecting objects in test_image_12
[2024-04-21 15:24:06,162]::[INFO] Query image IconDataset/png/022-car.png yields enough good matches - 17/10. Finding transform...
[2024-04-21 15:24:06,259]::[INFO] Query image IconDataset/png/015-barn.png yields enough good matches - 28/10. Finding transform...
[2024-04-21 15:24:06,574]::[INFO] Query image IconDataset/png/027-gas-station.png yields enough good matches - 12/10. Finding transform...


...

[2024-04-21 15:25:01,382]::[INFO] Detecting objects in test_image_20
[2024-04-21 15:25:01,703]::[INFO] Query image IconDataset/png/041-windmill.png yields enough good matches - 17/10. Finding transform...
[2024-04-21 15:25:04,110]::[INFO] Query image IconDataset/png/018-bank.png yields enough good matches - 21/10. Finding transform...
[2024-04-21 15:25:04,730]::[INFO] Query image IconDataset/png/030-telephone-booth.png yields enough good matches - 10/10. Finding transform...
[2024-04-21 15:25:05,268]::[INFO] Query image IconDataset/png/005-silo.png yields enough good matches - 27/10. Finding transform...


[2024-04-21 15:25:05,601]::[INFO] Detecting objects in test_image_7
[2024-04-21 15:25:07,597]::[INFO] Query image IconDataset/png/015-barn.png yields enough good matches - 11/10. Finding transform...
[2024-04-21 15:25:07,935]::[INFO] Query image IconDataset/png/027-gas-station.png yields enough good matches - 21/10. Finding transform...
[2024-04-21 15:25:08,332]::[INFO] Query image IconDataset/png/018-bank.png yields enough good matches - 21/10. Finding transform...
[2024-04-21 15:25:08,622]::[INFO] Query image IconDataset/png/021-solar-panel.png yields enough good matches - 15/10. Finding transform...
[2024-04-21 15:25:09,412]::[INFO] Query image IconDataset/png/014-flower.png yields enough good matches - 12/10. Finding transform...


[2024-04-21 15:25:09,832]::[INFO] Detecting objects in test_image_8
[2024-04-21 15:25:10,155]::[INFO] Query image IconDataset/png/041-windmill.png yields enough good matches - 25/10. Finding transform...
[2024-04-21 15:25:10,325]::[INFO] Query image IconDataset/png/037-post-office.png yields enough good matches - 17/10. Finding transform...


[2024-04-21 15:25:14,078]::[INFO] Detecting objects in test_image_16
[2024-04-21 15:25:14,250]::[INFO] Query image IconDataset/png/009-airport.png yields enough good matches - 56/41. Finding transform...
[2024-04-21 15:28:41,386]::[INFO] Query image IconDataset/png/033-hydrant.png yields enough good matches - 54/41. Finding transform...
[2024-04-21 15:39:09,809]::[INFO] Query image IconDataset/png/041-windmill.png yields enough good matches - 119/41. Finding transform...
[2024-04-21 15:39:19,814]::[INFO] Query image IconDataset/png/025-factory.png yields enough good matches - 47/41. Finding transform...
[2024-04-21 15:41:03,395]::[INFO] Query image IconDataset/png/037-post-office.png yields enough good matches - 63/41. Finding transform...
[2024-04-21 15:57:07,276]::[INFO] Query image IconDataset/png/040-bus-stop.png yields enough good matches - 53/41. Finding transform...


KeyboardInterrupt: 